In [1]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-03-11 16:32:28.489923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 16:32:29.375828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2024-03-11 16:32:29.957200: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 16:32:30.081647: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
dir_path = "./archive/chest_xray/chest_xray"

In [4]:
''' training path '''
train_p = os.path.join(dir_path, "train")

''' test path '''
test_p =os.path.join(dir_path, "test")

''' val path '''
val_p = os.path.join(dir_path, "val")

In [5]:
''' PNEUMONIA images '''
p_train_images = glob.glob(train_p + "/PNEUMONIA/*.jpeg")

''' NORMAL  images '''
n_train_images = glob.glob(train_p + "/NORMAL/*.jpeg")

In [6]:
df = pd.DataFrame(np.concatenate([[0]*len(n_train_images) , [1] *  len(p_train_images)]),columns=["class"])

In [7]:
""" sns.countplot(df['class'],data=df) """

" sns.countplot(df['class'],data=df) "

In [8]:
''' Data Augmentation '''
train_ds = ImageDataGenerator(rescale =1/255, shear_range=10, zoom_range = 0.2, horizontal_flip = True, 
                              width_shift_range=0.2, fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [9]:
''' VGG16 Model '''

model1 = VGG16(include_top = False,weights='imagenet')

''' freezing layers '''
for i in model1.layers:
    i.trainable=False
    
out = model1.output

''' GlobalAveragePooling Layer '''
out = keras.layers.GlobalAveragePooling2D()(out)
out = keras.layers.Dense(128, activation='relu')(out)

''' Classification Layer '''
pred = keras.layers.Dense(1, activation='sigmoid')(out)

''' final model '''
model= keras.Model(model1.input, pred)

lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' Compile the model '''
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(lr_schedule), metrics=['accuracy'])

In [10]:
''' lets see what the model looks like '''
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,780,481 (56.38 MB)

 Trainable params: 65,793 (257.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
''' flow from directory '''
train_ds = train_ds.flow_from_directory(train_p, target_size=(150,150), batch_size= 16, class_mode='binary')

val_ds = val_datagen.flow_from_directory(val_p, target_size=(150,150),batch_size=8, class_mode='binary')

test_ds = val_datagen.flow_from_directory(test_p, target_size=(150,150), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
''' training '''
history = model.fit(train_ds, steps_per_epoch = 100, epochs=20, validation_data=val_ds)

Epoch 1/20


/home/mohamed/Desktop/Projects/Mini-Projects-Python/Machine Learning/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 54s 522ms/step - accuracy: 0.6806 - loss: 5.7908 - val_accuracy: 0.8125 - val_loss: 0.5160
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 508ms/step - accuracy: 0.8192 - loss: 0.4804 - val_accuracy: 0.7500 - val_loss: 0.4355
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 500ms/step - accuracy: 0.8581 - loss: 0.3631 - val_accuracy: 0.6875 - val_loss: 0.9806
Epoch 4/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 37s 510ms/step - accuracy: 0.8955 - loss: 0.2707

2024-03-11 16:35:19.176630: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 135ms/step - accuracy: 0.8981 - loss: 0.2567 - val_accuracy: 0.6875 - val_loss: 0.6778
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 505ms/step - accuracy: 0.8747 - loss: 0.3177 - val_accuracy: 0.6875 - val_loss: 0.8052
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 506ms/step - accuracy: 0.8621 - loss: 0.3446 - val_accuracy: 0.8750 - val_loss: 0.3385
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 504ms/step - accuracy: 0.9046 - loss: 0.2469 - val_accuracy: 0.8750 - val_loss: 0.2247
Epoch 8/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 36s 494ms/step - accuracy: 0.8641 - loss: 0.3194

...

100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - accuracy: 0.8936 - loss: 0.2357 - val_accuracy: 0.8125 - val_loss: 0.3604
Epoch 13/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 505ms/step - accuracy: 0.9036 - loss: 0.2680 - val_accuracy: 0.9375 - val_loss: 0.1808
Epoch 14/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 503ms/step - accuracy: 0.9198 - loss: 0.2148 - val_accuracy: 0.8750 - val_loss: 0.3057
Epoch 15/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 50s 503ms/step - accuracy: 0.9059 - loss: 0.2171 - val_accuracy: 0.9375 - val_loss: 0.2204
Epoch 16/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 36s 498ms/step - accuracy: 0.9195 - loss: 0.2043

2024-03-11 16:43:37.260545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step - accuracy: 0.9186 - loss: 0.2116 - val_accuracy: 0.9375 - val_loss: 0.2015
Epoch 17/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 504ms/step - accuracy: 0.9097 - loss: 0.2635 - val_accuracy: 0.9375 - val_loss: 0.2093
Epoch 18/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 507ms/step - accuracy: 0.9275 - loss: 0.2097 - val_accuracy: 0.8125 - val_loss: 0.3457
Epoch 19/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 51s 506ms/step - accuracy: 0.8995 - loss: 0.2432 - val_accuracy: 0.9375 - val_loss: 0.2487
Epoch 20/20
 26/100 ━━━━━━━━━━━━━━━━━━━━ 37s 501ms/step - accuracy: 0.9415 - loss: 0.1522

2024-03-11 16:46:23.586474: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 132ms/step - accuracy: 0.9332 - loss: 0.1728 - val_accuracy: 0.9375 - val_loss: 0.1965


In [15]:
''' evaluating model '''
accuracy = model.evaluate(test_ds)[1]

/home/mohamed/Desktop/Projects/Mini-Projects-Python/Machine Learning/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 978ms/step - accuracy: 0.9200 - loss: 0.2568


In [17]:
accuracy

0.9006410241127014

In [18]:
''' unfreezing layers '''
for layer in model1.layers:
    layer.trainable = True

In [19]:
'''Finetuning the model'''
lr = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' compile the model '''
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule), loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [21]:
''' training '''
model.fit(train_ds, steps_per_epoch = 50, epochs=100, validation_data=val_ds)

Epoch 1/100


/home/mohamed/Desktop/Projects/Mini-Projects-Python/Machine Learning/.venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:669: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


50/50 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - binary_accuracy: 0.9018 - loss: 0.4871 - val_binary_accuracy: 0.9375 - val_loss: 0.2582
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - binary_accuracy: 0.9036 - loss: 0.2173 - val_binary_accuracy: 0.8125 - val_loss: 0.2469
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - binary_accuracy: 0.8898 - loss: 0.2461 - val_binary_accuracy: 0.7500 - val_loss: 0.5269
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - binary_accuracy: 0.9474 - loss: 0.1511 - val_binary_accuracy: 0.7500 - val_loss: 0.6448
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - binary_accuracy: 0.9489 - loss: 0.1720 - val_binary_accuracy: 0.7500 - val_loss: 0.5784
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - binary_accuracy: 0.9269 - loss: 0.2033 - val_binary_accuracy: 0.8750 - val_loss: 0.2463
Epoch 7/100
26/50 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - binary_accuracy: 0.9332 - loss: 0.1455

2024-03-11 17:03:52.170115: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


50/50 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - binary_accuracy: 0.9341 - loss: 0.1458 - val_binary_accuracy: 0.8125 - val_loss: 0.4909
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - binary_accuracy: 0.9334 - loss: 0.1728 - val_binary_accuracy: 0.9375 - val_loss: 0.1565
Epoch 9/100
17/50 ━━━━━━━━━━━━━━━━━━━━ 1:08 2s/step - binary_accuracy: 0.9558 - loss: 0.1540

KeyboardInterrupt: 